In [6]:
from typing import Callable, List, Union

import numpy as np

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem


In [124]:
import os
os.path.join(os.curdir,"data/mono.csv")

'.\\data/mono.csv'

In [3]:
Molecule = Union[str, Chem.Mol]
FeaturesGenerator = Callable[[Molecule], np.ndarray]

In [13]:
len(a)

639

In [126]:
import pandas as pd
a = pd.read_csv("./data/mono.csv")
b = dict([(a['smiles'][i], a.iloc[i].values[1:]) for i in range(len(a))])


In [130]:
b["NC(Cc1cc(I)c(Oc2cc(I)c(O)c(I)c2)c(I)c1)C(=O)O".upper()]

KeyError: 'NC(CC1CC(I)C(OC2CC(I)C(O)C(I)C2)C(I)C1)C(=O)O'

In [131]:
np.linspace(0, 0, 3)

array([0., 0., 0.])

In [9]:
import csv
from tqdm import tqdm

with open(path:="./data/test_random.csv") as f:
    reader = csv.DictReader(f)
    if target_columns is None:
        target_columns = get_task_names(
            path=path,
            smiles_columns=smiles_columns,
            target_columns=target_columns,
            ignore_columns=ignore_columns,
        )

    all_smiles, all_targets, all_rows, all_features, all_phase_features, all_weights = [], [], [], [], [], []
    for i, row in enumerate(tqdm(reader)):
        smiles = [row[c] for c in ['smiles_1', 'smiles_2']]

        targets = [float(row[column]) if row[column] not in ['','nan'] else None for column in target_columns]

        # Check whether all targets are None and skip if so
        if skip_none_targets and all(x is None for x in targets):
            continue

        all_smiles.append(smiles)
        all_targets.append(targets)

        if features_data is not None:
            all_features.append(features_data[i])

        if phase_features is not None:
            all_phase_features.append(phase_features[i])

        if data_weights is not None:
            all_weights.append(data_weights[i])

        if store_row:
            all_rows.append(row)

        if len(all_smiles) >= max_data_size:
            break

    atom_features = None
    atom_descriptors = None
    if args is not None and args.atom_descriptors is not None:
        try:
            descriptors = load_valid_atom_or_bond_features(atom_descriptors_path, [x[0] for x in all_smiles])
        except Exception as e:
            raise ValueError(f'Failed to load or validate custom atomic descriptors or features: {e}')

        if args.atom_descriptors == 'feature':
            atom_features = descriptors
        elif args.atom_descriptors == 'descriptor':
            atom_descriptors = descriptors

    bond_features = None
    if args is not None and args.bond_features_path is not None:
        try:
            bond_features = load_valid_atom_or_bond_features(bond_features_path, [x[0] for x in all_smiles])
        except Exception as e:
            raise ValueError(f'Failed to load or validate custom bond features: {e}')

    data = MoleculeDataset([
        MoleculeDatapoint(
            smiles=smiles,
            targets=targets,
            row=all_rows[i] if store_row else None,
            data_weight=all_weights[i] if data_weights is not None else 1.,
            features_generator=features_generator,
            features=all_features[i] if features_data is not None else None,
            phase_features=all_phase_features[i] if phase_features is not None else None,
            atom_features=atom_features[i] if atom_features is not None else None,
            atom_descriptors=atom_descriptors[i] if atom_descriptors is not None else None,
            bond_features=bond_features[i] if bond_features is not None else None,
            overwrite_default_atom_features=args.overwrite_default_atom_features if args is not None else False,
            overwrite_default_bond_features=args.overwrite_default_bond_features if args is not None else False
        ) for i, (smiles, targets) in tqdm(enumerate(zip(all_smiles, all_targets)),
                                           total=len(all_smiles))
    ])

6347it [00:00, 9920.20it/s]


In [14]:
a="C[C@@H]1C[C@H]2C3CCC4=CC(=O)C=C[C@@]4([C@]3([C@H](C[C@@]2([C@]1(C(=O)CCl)O)C)O)Cl)C"
a.split(">")[0]

'C[C@@H]1C[C@H]2C3CCC4=CC(=O)C=C[C@@]4([C@]3([C@H](C[C@@]2([C@]1(C(=O)CCl)O)C)O)Cl)C'

In [108]:
# 三对drugs
mol_vecs1 = [torch.from_numpy(np.array([1,2,3,4,5])),
            torch.from_numpy(np.array([6,7,8,9,10])),
            torch.from_numpy(np.array([11,12,13,14,15]))]
mol_vecs2 = [torch.from_numpy(np.array([5,4,3,2,1])),
            torch.from_numpy(np.array([10,9,8,7,6])),
            torch.from_numpy(np.array([15,14,13,12,11]))]

encodings = [torch.stack(mol_vecs1, dim=0).float(), 
             torch.stack(mol_vecs2, dim=0).float()]
encodings

[tensor([[ 1.,  2.,  3.,  4.,  5.],
         [ 6.,  7.,  8.,  9., 10.],
         [11., 12., 13., 14., 15.]]),
 tensor([[ 5.,  4.,  3.,  2.,  1.],
         [10.,  9.,  8.,  7.,  6.],
         [15., 14., 13., 12., 11.]])]

In [107]:
features_batch=[[[1,2],[3,4]],
                [[5,6],[7,8]],
                [[9,10],[11,12]]]
test = [[features[i] for features in features_batch] for i in range(len(features_batch[0]))]

features_batch = torch.from_numpy(np.stack(features_batch)).float()
test = [torch.from_numpy(np.stack(test)[0]).float(),
        torch.from_numpy(np.stack(test)[1]).float()]
test

[tensor([[ 1.,  2.],
         [ 5.,  6.],
         [ 9., 10.]]),
 tensor([[ 3.,  4.],
         [ 7.,  8.],
         [11., 12.]])]

In [117]:
a=[torch.cat([encoding, batch], dim=1) for encoding, batch in zip(encodings, test)]
a

[tensor([[ 1.,  2.,  3.,  4.,  5.,  1.,  2.],
         [ 6.,  7.,  8.,  9., 10.,  5.,  6.],
         [11., 12., 13., 14., 15.,  9., 10.]]),
 tensor([[ 5.,  4.,  3.,  2.,  1.,  3.,  4.],
         [10.,  9.,  8.,  7.,  6.,  7.,  8.],
         [15., 14., 13., 12., 11., 11., 12.]])]

In [116]:
from functools import reduce
output = reduce(lambda x, y: x+y-x*y, a)
output

tensor([[   1.,   -2.,   -3.,   -2.,    1.,    1.,   -2.],
        [ -44.,  -47.,  -48.,  -47.,  -44.,  -23.,  -34.],
        [-139., -142., -143., -142., -139.,  -79.,  -98.]])